In [1]:
#安装第三方包
pip install pandas openpyxl numpy pyperclip

Note: you may need to restart the kernel to use updated packages.


In [1]:
# 导入
import pandas as pd
from datetime import datetime
import re
from openpyxl.utils import get_column_letter
from pandas import ExcelWriter
import numpy as np
import pyperclip
import os

In [5]:
# 输入上次和本次采集数据的日期
datetime_last = datetime(2025,7,15) # 上次采集数据的日期
datetime_this = datetime(2025,7,16) # 本次采集数据的日期

# 构造文件路径
def get_file_paths(date):
    date_str = date.strftime('%Y-%m-%d')
    raw_path = f'raw_data/calculate-{date_str}.xlsx'
    save_path = f'raw_data/proceed/calculate-{date_str}.xlsx'
    return raw_path, save_path

# 创建保存目录（如果不存在）
os.makedirs('raw_data/proceed', exist_ok=True)

# 处理单个文件
def process_file(input_path, output_path):
    # 读取文件
    df = pd.read_excel(input_path)

    # 2.1 去除前两列
    df = df.iloc[:, 2:]

    # 2.2 去除第三列中的所有“#”，并转换为数值型以便正确排序
    if df.shape[1] >= 1:
        df.iloc[:, 0] = df.iloc[:, 0].astype(str).str.replace('#', '', regex=False)
        df.iloc[:, 0] = pd.to_numeric(df.iloc[:, 0], errors='coerce')  # 转成数字，非法项为NaN

    # 2.3 根据第三列排序（现在是第一列）
    df = df.sort_values(by=df.columns[0], ascending=True)

    # 2.4 去除重复行
    df = df.drop_duplicates()
    
    # 2.5 将最后一列中的空值替换为 0
    if df.shape[1] >= 1:
        df.iloc[:, -1] = df.iloc[:, -1].fillna(0)
    
    # 保存结果
    df.to_excel(output_path, index=False)

# 处理两个文件
for dt in [datetime_last, datetime_this]:
    input_file, output_file = get_file_paths(dt)
    process_file(input_file, output_file)

print("文件处理完毕。")


C:\Users\19872\AppData\Local\Temp\ipykernel_32052\2005464941.py:26: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.iloc[:, 0] = pd.to_numeric(df.iloc[:, 0], errors='coerce')  # 转成数字，非法项为NaN


文件处理完毕。


C:\Users\19872\AppData\Local\Temp\ipykernel_32052\2005464941.py:26: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.iloc[:, 0] = pd.to_numeric(df.iloc[:, 0], errors='coerce')  # 转成数字，非法项为NaN


In [12]:
# 设置用户名
username = '斯格里芬'

# 构造文件路径
def get_file_path(date):
    date_str = date.strftime('%Y-%m-%d')
    return f'raw_data/proceed/calculate-{date_str}.xlsx'

# 读取数据
df_last = pd.read_excel(get_file_path(datetime_last))
df_this = pd.read_excel(get_file_path(datetime_this))

# 按用户名拆分
def split_by_username(df, username_column='author'):
    if username_column not in df.columns:
        raise ValueError(f"列 '{username_column}' 不存在，请检查 Excel 文件。")
    return df[df[username_column] == username], df[df[username_column] != username]

df_last_same, df_last_diff = split_by_username(df_last)
df_this_same, df_this_diff = split_by_username(df_this)

# 打印当天总体概览
print("===== 分析报告 =====")
print(f"在 {datetime_this.date()} 的帖子中")
total_rows   = len(df_this)
same_rows    = len(df_this_same)
diff_rows    = len(df_this_diff)
total_reco   = df_this.iloc[:, -1].sum()
same_reco    = df_this_same.iloc[:, -1].sum()
diff_reco    = df_this_diff.iloc[:, -1].sum()

print(f"一共有 {total_rows} 条回复。")
print(f"其中楼主更新了 {same_rows} 条，")
print(f"观众回复了 {diff_rows} 条，")
print(f"总推荐数为 {total_reco}，")
print(f"楼主获得推荐为 {same_reco}，")
print(f"观众回复获得推荐为 {diff_reco}")

print(f"\n相比于 {datetime_last.date()}：")

def analyze_diff(df1, df2, label, is_reply=False):
    # 1) 行数差异
    delta_rows = len(df2) - len(df1)
    if is_reply:
        print(f"[回复] 增加了 {delta_rows} 回复。")
    else:
        print(f"[更新] 楼主更新了 {delta_rows} 次。")

    # 2) 推荐数总变化
    delta_reco = df2.iloc[:, -1].sum() - df1.iloc[:, -1].sum()
    if is_reply:
        print(f"[回复推荐] 增加了 {delta_reco} 推荐。")
    else:
        print(f"[推荐] 增加了 {delta_reco} 推荐。")

    # 3) 共同行的推荐差值
    cols = list(df1.columns[:-1])
    df1_idx = df1.set_index(cols)
    df2_idx = df2.set_index(cols)
    common = df1_idx.index.intersection(df2_idx.index)

    if len(common) > 0:
        diffs = df2_idx.loc[common].iloc[:, -1] - df1_idx.loc[common].iloc[:, -1]
        positive = diffs[diffs > 0]
        if not positive.empty:
            tag = "回复" if is_reply else "更新楼层"
            print(f"[获得推荐] 发生推荐变化的{tag}：")
            for idx, inc in positive.items():
                floor = idx[0] if isinstance(idx, tuple) else idx
                print(f"  第 {floor} 楼获得了 {inc} 推荐")
        else:
            print(f"[获得推荐] 所有回复都没有获得推荐")
    else:
        print(f"[获得推荐] 无可比较的相同行")

    # 4) 新增行的推荐总和及楼层列表
    new_entries = set(df2_idx.index) - set(df1_idx.index)
    if new_entries:
        added_sum = 0
        floors = []
        for key in new_entries:
            val = df2_idx.loc[key].iloc[-1]
            if val > 0:
                added_sum += val
                # 提取楼层号（索引的第一个元素）
                floor = key[0] if isinstance(key, tuple) else key
                floors.append(floor)
        if added_sum > 0:
            if is_reply:
                print(f"[新增回复推荐] 新增回复获得了 {added_sum} 推荐，楼层：{', '.join(map(str, floors))}")
            else:
                print(f"[新增更新推荐] 新增更新获得了 {added_sum} 推荐，楼层：{', '.join(map(str, floors))}")

# 分析楼主部分
analyze_diff(df_last_same, df_this_same, label="楼主", is_reply=False)
# 分析回复部分
analyze_diff(df_last_diff, df_this_diff, label="回复", is_reply=True)

===== 分析报告 =====
在 2025-07-16 的帖子中
一共有 319 条回复。
其中楼主更新了 234 条，
观众回复了 85 条，
总推荐数为 816，
楼主获得推荐为 726，
观众回复获得推荐为 90

相比于 2025-07-15：
[更新] 楼主更新了 1 次。
[推荐] 增加了 2 推荐。
[获得推荐] 发生推荐变化的更新楼层：
  第 317 楼获得了 1 推荐
[新增更新推荐] 新增更新获得了 1 推荐，楼层：318
[回复] 增加了 0 回复。
[回复推荐] 增加了 0 推荐。
[获得推荐] 所有回复都没有获得推荐
